<a href="https://colab.research.google.com/github/saeu5407/daily_tensorflow/blob/main/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텐서플로우 튜토리얼
### 기초 텍스트 분류기 생성 예제
주석달면서 조금 더 다듬기

텐서플로우 원본 링크 : https://www.tensorflow.org/tutorials/keras/text_classification?hl=ko

In [1]:
import matplotlib.pyplot as plt
import os
import re
import shutil # 파일, 폴더 복사용 패키지, 쉘 유틸리티
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

### 데이터 전처리 샘플

In [2]:
# 다운로드 URL
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

# 데이터셋 다운로드
dataset = tf.keras.utils.get_file("aclImdb_v1", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')
# 데이터셋 경로
dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

# TRAIN SET 경로
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

# 트레이닝 셋 중 샘플 경로
sample_file = os.path.join(train_dir, 'pos/1181_9.txt')
with open(sample_file) as f:
  print(f.read())

Rachel Griffiths writes and directs this award winning short film. A heartwarming story about coping with grief and cherishing the memory of those we've loved and lost. Although, only 15 minutes long, Griffiths manages to capture so much emotion and truth onto film in the short space of time. Bud Tingwell gives a touching performance as Will, a widower struggling to cope with his wife's death. Will is confronted by the harsh reality of loneliness and helplessness as he proceeds to take care of Ruth's pet cow, Tulip. The film displays the grief and responsibility one feels for those they have loved and lost. Good cinematography, great direction, and superbly acted. It will bring tears to all those who have lost a loved one, and survived.


In [3]:
# 뽑아온 데이터셋에 필요없는 데이터들이 있어 제거
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir) # shutil을 사용하는 모습

In [4]:
# text_dataset_from_directory 사용하여 데이터 로드 및 분할
"""
text_dataset_from_directory : 폴더별로 정리되어 있는 텍스트 데이터를 로드, 분할하는 함수(이미지데이터에도 비슷한거 있음)

tf.keras.utils.text_dataset_from_directory(
    directory,              # 디렉토리 경로
    labels='inferred',      # 라벨링을 어떻게 할 지 / inferred는 폴더명 참고
    label_mode='int',       # 라벨 인코딩 방식
    class_names=None,       # labels='inferred'일 때 사용, 클래스 순서 제어용
    batch_size=32,          # 배치 사이즈
    max_length=None,        # 단어 최대길이
    shuffle=True,           # 섞을지 여부
    seed=None,              # 일 때 무작위 시드
    validation_split=None,  # 검증 세트 스플릿 여부
    subset=None,            # 'training' 또는 'validation' 세트 여부
    follow_links=False      # 하위 디렉토리까지 찾아보는지 여부? 정확하지 않음
)
"""
batch_size = 32
seed = 42 # 랜덤 시드와 배치사이즈를 고정하는 모습

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='training', # 트레이닝셋이라는 걸 명시
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [5]:
# 샘플 프린트
for text_batch, label_batch in raw_train_ds.take(1): # take로 배치 하나를 뽑아낼 수 있는 것으로 보임
  for i in range(3):
    print("Review", text_batch.numpy()[i])
    print("Label", label_batch.numpy()[i])

Review b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
Label 0
Review b"David Mamet is a very interesting and a very un-equal director. His first movie 'House of Games' was the one I liked best, and it set a series of films with characters whose perspective of life changes as they get into 

In [6]:
# 긍,부정 레이블별 클래스명
print("Label 0 corresponds to", raw_train_ds.class_names[0])
print("Label 1 corresponds to", raw_train_ds.class_names[1])

Label 0 corresponds to neg
Label 1 corresponds to pos


In [7]:
# 검증 데이터 세트 생성
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='validation', # validataion set 명시
    seed=seed)

# 테스트 데이터 세트 생성
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/test', 
    batch_size=batch_size) # test set은 따로 명시하지 않음, validation_split이 필요할 때만 해당 옵션을 쓰는 것으로 보임.

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [8]:
# 테스트 데이터 전처리

# 토큰 등 제거하여 데이터 정제
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data) # 모두 소문자로 변경
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ') # <br />등을 제거
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), # string.punctuation : 구두점을 의미합니다. # re.escape : 이스케이프 처리 \처럼.
                                  '')
"""
위의 '[%s]' % re.escape(string.punctuation) 쪽이 이해가 안갔는데,
사실상 % 의 기능이 .format()과 같다고 한다.
즉 위의 뜻은 '{%s}'.format(re.escape(string.punctuation)) 과 같다.
뒤에 저 단어들만 뽑아오겠다는 뜻.
"""

"\n위의 '[%s]' % re.escape(string.punctuation) 쪽이 이해가 안갔는데,\n사실상 % 의 기능이 .format()과 같다고 한다.\n즉 위의 뜻은 '{%s}'.format(re.escape(string.punctuation)) 과 같다.\n뒤에 저 단어들만 뽑아오겠다는 뜻.\n"

In [9]:
# custom_standardization 샘플
print(text_batch.numpy()[0])
print(custom_standardization(text_batch.numpy()[0]))

b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
tf.Tensor(b'pandemonium is a horror movie spoof that comes off more stupid than funny believe me when i tell you i love comedies especially comedy spoofs airplane the naked gun trilogy blazing saddles high anxiety and spaceballs are some of my

In [10]:
# Text Vectorization 수행
# 표준화, 토큰화, 벡터화를 수행합니다.
max_features = 10000
sequence_length = 250

vectorize_layer = TextVectorization(
    standardize=custom_standardization, # standardize 즉 표준화를 우리가 만든 함수를 사용하여 작업한다는 뜻
    max_tokens=max_features, # 어휘의 최대 크기, 단어 사전 수
    output_mode='int',
    output_sequence_length=sequence_length) # 시퀀스의 최대 길이

In [11]:
# 전처리 레이어의 상태를 Train Set에 맞추기
# 훈련 전에 적응(Adaptation)하기 위해 쓰기에 adapt이라고 부르는 메서드를 사용합니다.
# 해당 메서드는 Test set 등이 아닌 Train set에만 적용해서 Data Leakeage를 막아야 합니다.
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [12]:
# 이에 대한 샘플
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

# retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

Review tf.Tensor(b'Great movie - especially the music - Etta James - "At Last". This speaks volumes when you have finally found that special someone.', shape=(), dtype=string)
Label neg
Vectorized review (<tf.Tensor: shape=(1, 250), dtype=int64, numpy=
array([[  86,   17,  260,    2,  222,    1,  571,   31,  229,   11, 2418,
           1,   51,   22,   25,  404,  251,   12,  306,  282,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
       

In [13]:
print("1287 ---> ",vectorize_layer.get_vocabulary()[1287])
print(" 313 ---> ",vectorize_layer.get_vocabulary()[313])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

1287 --->  silent
 313 --->  night
Vocabulary size: 10000


In [14]:
# 데이터 전처리 수행
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [15]:
### 추가 ###
# I/O가 차단되지 않도록 데이터를 로드할 때 사용해야 하는 두 가지 중요한 메서드가 존재합니다.

# .cache() : 데이터가 디스크에서 로드된 후 메모리에 데이터를 보관하는 메서드. 이렇게 하면 모델을 훈련하는 동안 데이터세트로 인해 병목 현상이 발생하지 않습니다. 
# 데이터세트가 너무 커서 메모리에 맞지 않는 경우, 이 메서드를 사용하여 성능이 뛰어난 온 디스크 캐시를 생성할 수도 있습니다. 많은 작은 파일보다 읽기가 더 효율적입니다.

# .prefetch() : 훈련 중에 다음 배치 스텝의 데이터를 읽어옵니다. 속도 향상.
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

### 모델링

In [16]:
embedding_dim = 16

model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim), 
  # Embedding 층은 정수로 인코딩된 단어를 입력 받고 각 단어 인덱스에 해당하는 임베딩 벡터를 찾습니다. 
  # 이 벡터는 모델이 훈련되면서 학습됩니다. 이 벡터는 출력 배열에 새로운 차원으로 추가됩니다. 최종 차원은 (batch, sequence, embedding)이 됩니다.
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  # GlobalAveragePooling1D 층은 sequence 차원에 대해 평균을 계산하여 각 샘플에 대해 고정된 길이의 출력 벡터를 반환합니다. 
  # 길이가 다른 입력을 다루는 가장 간단한 방법이라고 합니다.
  layers.Dropout(0.2),
  layers.Dense(1)]) # 최종 16개 레이어를 1개로 줄이는 FC레이어로 마무리합니다.

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160016    
_________________________________________________________________
dropout (Dropout)            (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 160,033
Trainable params: 160,033
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [20]:
# 훈련 단계
history = model.fit(train_ds,
                    epochs=40,
                    batch_size=512,
                    verbose=1)

Epoch 1/40
625/625 [==============================] - 11s 15ms/step - loss: 0.6641 - binary_accuracy: 0.6963
Epoch 2/40
625/625 [==============================] - 4s 7ms/step - loss: 0.5509 - binary_accuracy: 0.8015
Epoch 3/40
625/625 [==============================] - 4s 7ms/step - loss: 0.4473 - binary_accuracy: 0.8447
Epoch 4/40
625/625 [==============================] - 4s 7ms/step - loss: 0.3801 - binary_accuracy: 0.8641
Epoch 5/40
625/625 [==============================] - 4s 7ms/step - loss: 0.3369 - binary_accuracy: 0.8794
Epoch 6/40
625/625 [==============================] - 4s 6ms/step - loss: 0.3047 - binary_accuracy: 0.8895
Epoch 7/40
625/625 [==============================] - 4s 7ms/step - loss: 0.2820 - binary_accuracy: 0.8957
Epoch 8/40
625/625 [==============================] - 4s 7ms/step - loss: 0.2631 - binary_accuracy: 0.9039
Epoch 9/40
625/625 [==============================] - 4s 6ms/step - loss: 0.2457 - binary_accuracy: 0.9118
Epoch 10/40
625/625 [==============

In [21]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

782/782 [==============================] - 8s 10ms/step - loss: 0.5336 - binary_accuracy: 0.8463
Loss:  0.5335999727249146
Accuracy:  0.8462799787521362


In [24]:
# 모델 내보내기
export_model = tf.keras.Sequential([
  vectorize_layer,
  model,
  layers.Activation('sigmoid')
])

export_model.compile(
    loss=losses.BinaryCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)

# Test it with `raw_test_ds`, which yields raw strings
loss, accuracy = export_model.evaluate(raw_test_ds)
print(accuracy)

782/782 [==============================] - 8s 10ms/step - loss: 0.5336 - accuracy: 0.8463
0.8462799787521362


In [25]:
# 새로운 데이터로 추론
examples = [
  "The movie was great!",
  "The movie was okay.",
  "The movie was terrible..."
]

export_model.predict(examples)

array([[0.70060325],
       [0.32022655],
       [0.22488159]], dtype=float32)